# Pitch sequences embedded in scale changes

## Deriving pitch class sequences from a series of pitch class sets

When writing a piece of music that contains a sequence of chord or key changes, you often want to write parts that source pitches from said chords/keys. 
For example, if you have a 4 bar loop that goes `C, G, Dm, F`, then you may want to have a bass line that goes `C, G, D, F`, or `E, D, F, F`, or `G, B, A, A`, etc...

One can account for the set of all possible such sequences by taking the cartesian product of the pitch class sets.

In our example, the pitch class set sequence would be 

```
[
    {0,4,7} mod 12 root 0, 
    {2,7,11} mod 12 root 7, 
    {2,5,9} mod 12 root 2,
    {0,5,9} mod 12 root 5
]
```

From this sequence of pitch class sets, we can pick one pitch class from each set, giving us sequences like `[0,2,2,0]`, `[0,2,2,5]`, etc...

In [ ]:
import itertools
from harmonica.pitch import PitchClassSet


def pcseqs_from_pcset_seq(pcset_seq: list[PitchClassSet]) -> list[list[int]]:
    pcseqs = []

    assert all(
        [pcset.modulus == pcset_seq[0].modulus for pcset in pcset_seq]
    ), "Pitch class sets must all have the same modulus."

    modulus = pcset_seq[0].modulus
    pcsets = [pcset.pitch_classes for pcset in pcset_seq]

    for pcseq in itertools.product(*pcsets):
        pcseqs.append(list(pcseq))

    return pcseqs


pcs = [PitchClassSet(x, 12) for x in [[0, 4, 7], [2, 7, 11], [2, 5, 9], [0, 5, 9]]]

pcseqs = pcseqs_from_pcset_seq(pcs)

for pcseq in pcseqs:
    print(pcseq)

## Deriving pitch sequences from pitch class sequences

From any given pitch class sequence, we can derive infinitely many pitch sequences, because a pitch class represents every possible octave. 
There aren't infinitely many (audible) pitches, of course, so this isn't very useful. Therefore, we should aim to reduce this infinite set 
of pitch sequences to a finite set.

To do this, we must specify a range of octaves. For example, if we have a pitch class `7`, then all pitches with pitch class `7` 
between octaves 2 and 4 would be given by taking `12 * 2 + 7`, `12 * 3 + 7` and `12 * 4 + 7`, giving us 31, 43 and 55.

In [ ]:
def pcseq_to_pseqs(
    pcseq: list[int], mod: int, min_octave: int, max_octave: int
) -> list[list[int]]:
    pseqs = []

    next_list = [
        [mod * oct + pc for oct in range(min_octave, max_octave + 1)] for pc in pcseq
    ]

    for pseq in itertools.product(*next_list):
        pseqs.append(list(pseq))

    return pseqs


print(pcseq_to_pseqs([7, 2, 4], 12, 3, 5))

[[43, 38, 40], [43, 38, 52], [43, 38, 64], [43, 50, 40], [43, 50, 52], [43, 50, 64], [43, 62, 40], [43, 62, 52], [43, 62, 64], [55, 38, 40], [55, 38, 52], [55, 38, 64], [55, 50, 40], [55, 50, 52], [55, 50, 64], [55, 62, 40], [55, 62, 52], [55, 62, 64], [67, 38, 40], [67, 38, 52], [67, 38, 64], [67, 50, 40], [67, 50, 52], [67, 50, 64], [67, 62, 40], [67, 62, 52], [67, 62, 64]]


Wow, that is epic. Why don't we just create a clip where we listen through all of these sequences? Hell yeah.

Let's put chords on top too, for harmonic context. The pitch class sequence is `[7,2,4]`, so let's make the chords Em, G, C.

To be more specific about what we're doing here: let's listen to all pitch sequences that can be derived from this chord progression,
contained to a single octave of pitches.

In [6]:
from fractions import Fraction
from harmonica.pitch import PitchSet
from harmonica.time import Clip, NoteClip, Note

pseqs = []
for pcseq in pcseqs_from_pcset_seq(
    [
        PitchClassSet(pitch_classes, 12)
        for pitch_classes in [[4, 7, 11], [2, 7, 11], [0, 4, 7]]
    ]
):
    pseqs += pcseq_to_pseqs(pcseq, 12, 5, 5)

chords = [
    chord.invert(2)
    for chord in [
        PitchSet([4, 7, 11]) + 60,
        PitchSet([2, 7, 11]) + 60,
        PitchSet([0, 4, 7]) + 60,
    ]
]

bass_clip = NoteClip([]).set_program(34)
chord_clip = NoteClip([]).set_program(5)

onset = Fraction(0)

for pseq in pseqs:
    for i, pitch in enumerate(pseq):
        bass_clip.add_event(Note(pitch=pitch, onset=onset, duration=Fraction(1)))
        chord_clip.add_event(chords[i].to_clip(onset=onset, duration=Fraction(1)))
        onset += Fraction(1)
    onset += Fraction(1)

Clip([bass_clip, chord_clip]).preview(tempo=55)

### A problem with this approach

This is really neat. Big fan of enumerating things. There's one problem though, which is that we're basically picking a range 
of pitches, like 60 - 71, and enumerating all pitch sequences with pitches in that range. This isn't very intuitive in terms of voice leading. 

I mean, just look at our results. The pitch class sequence `[4,11,0]`, for example. When we derive pitch sequences from this and are only 
considering a fixed range of 60 through 71, we get sequences like `[64,71,60]`, but what about `[64,71,72]`? What about `[64,59,60]`? 

The reason this kinda sucks is because we're getting an incomplete account of the intervallic "shapes" that can be derived from a given 
pitch class sequence. Intervals are a _very important feature_ of any pitch structure, so if our goal is to capture entire swathes of musical possibilities, 
then we should aim to enumerate all possible intervallic structures from a sequence of pitch classes.

### The cooler approach

Here's what you do. You take your pitch class sequence, like `[4,11,0]`.

You start with the first pitch class, 4. You pick an octave, or register, or whatever you want to call it - y'know, `12x + 4`. 
28, 40, 52, whatever. Let's just stick with 4 to keep things simple.

Now, lets consider the movement from 4 to 11. When moving from one pitch class to another, you have two parameters: 

1. Direction
2. Magnitude

When moving from 4 to 11, you can move either up or down. Then, you can add some amount of octaves to the movement. 

So you can move up to 11, 23, 35... Or, you can move down to -1, -13, -25...

The only exception to this is when a pitch class moves to itself, in which case you have another choice aside from moving up or down, 
which is to not move at all. +/- 0.

### The new algorithm

In the previous algorithm, we specified a range of octaves that we sampled all of our pitches from. In this new algorithm, we only specify the octave of 
the first pitch. Think of this algorithm as a lot more "relative" or "polar" than the previous one.

Another parameter will be the maximum magnitude, also specified in terms of octave. With a max magnitude of 0, we will only yield the pitches immediately 
above and below the previous pitch. That is to say, if the first pitch is 12, and the next pitch class is 4, and the max magnitude is 0, then the set of 
pitches we can sample from are either 4 or 16. If the max magnitude were 1, then we could sample from -8, 4, 16, or 28. And so on and so on...

Using this algorithm, we can enumerate every possible bass line, melody, pattern, WHATEVER! I mean, probably. What am I, a doctor?

In [ ]:
from harmonica.pitch import PitchSeq, PitchClassSet


"""
THIS ISNT DONE YET
"""


def pseqs_from_pcsets(
    pcset_seq: list[PitchClassSet], starting_octave: int, max_magnitude: int
) -> list[PitchSeq]:
    assert all(
        [pcset.modulus == pcset_seq[0].modulus for pcset in pcset_seq]
    ), "All pitch class sets in sequence must have the same modulus."

    assert max_magnitude >= 0, "Maximum octave magnitude must be non-negative."

    pseqs: list[PitchSeq] = []
    modulus = pcset_seq[0].modulus

    # Derive pitch class sequences from progression of pitch class sets.
    pcseqs = pcseqs_from_pcset_seq(pcset_seq)

    # Now we iterate through each of these pitch class sequences.
    for pcseq in pcseqs:
        # We add 12 * starting_octave to the first pitch class
        first_pitch_class = 12 * starting_octave + pcseq[0]
        # Iterate through intervals we can add to this pitch class
        for pc in pcseq[1:]:
            intervals = []

            if pc == first_pitch_class % modulus:
                intervals.append(0)

    return pseqs

NameError: name 'PitchClassSet' is not defined